In [11]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import psycopg2
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.metrics import accuracy_score
from mpl_toolkits.mplot3d import Axes3D
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix

In [12]:
# Establish a connection to the database
conn = psycopg2.connect(
    host="localhost",
    database="ProjetBI",
    user="postgres",
    password="marwa"
)
# Create a cursor object
cur = conn.cursor()

# Execute the SQL query
cur.execute("SELECT stats.\"Job_Duration\", stats.\"Data_Volume\", (select count(\"Component_PK\") from public.\"DimJob\" join public.\"DimComponent\" on(public.\"DimJob\".\"Job_PK\" = public.\"DimComponent\".\"Job_FK\") where \"Job_PK\"=stats.\"Job_FK\" group by \"Job_Code\") as \"Nbr_Components\" FROM public.\"FactStats\" stats")
# Fetch all the rows as a list of tuples
rows = cur.fetchall()

# Convert the list of tuples to a pandas DataFrame
df = pd.DataFrame(rows, columns=["Job_Duration","Data_Volume", "Nbr_Components"])
print(df)

       Job_Duration  Data_Volume  Nbr_Components
0             33447        39387               9
1             33447        39600               9
2             33447        39593               9
3             33447        39680               9
4             33447        41401               9
...             ...          ...             ...
11843         20993         3553               5
11844         20993         3554               5
11845         20993         3512               5
11846         20993         3482               5
11847         20993         3458               5

[11848 rows x 3 columns]


In [13]:
# normalize the data
scaler = MinMaxScaler()
df = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
print(df)

       Job_Duration  Data_Volume  Nbr_Components
0          1.000000     0.388254        1.000000
1          1.000000     0.390353        1.000000
2          1.000000     0.390284        1.000000
3          1.000000     0.391142        1.000000
4          1.000000     0.408107        1.000000
...             ...          ...             ...
11843      0.199666     0.035014        0.333333
11844      0.199666     0.035024        0.333333
11845      0.199666     0.034610        0.333333
11846      0.199666     0.034314        0.333333
11847      0.199666     0.034078        0.333333

[11848 rows x 3 columns]


In [14]:
#Train and test division
X2 = df.iloc[:,:-1]
y2 = df.iloc[:,-1]

X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.3, random_state=0)

In [15]:
#initialisation et entrainement du modèle
rf_model = RandomForestRegressor(n_estimators=100)
rf_model.fit(X_train2, y_train2)

#Test
y_pred2 = rf_model.predict(X_test2)

In [20]:
#R²
r2_random_forest = r2_score(y_test2, y_pred2)
print("R² : ",r2_random_forest)
mse = mean_squared_error(y_test2, y_pred2)
print("MSE:", mse)

R² :  0.9999375873454279
MSE: 4.128770120331218e-06
